# GENE EXPRESSION DATA

In [ ]:
!wget http://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip

--2025-04-19 00:42:57--  http://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip
Resolving www.cancerrxgene.org (www.cancerrxgene.org)... 193.62.203.106, 193.62.203.105, 2001:630:206:4::106, ...
Connecting to www.cancerrxgene.org (www.cancerrxgene.org)|193.62.203.106|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip [following]
--2025-04-19 00:42:58--  https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip
Connecting to www.cancerrxgene.org (www.cancerrxgene.org)|193.62.203.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142765551 (136M) [application/zip]
Saving to: ‘Cell_line_RMA_proc_basalExp.txt.zip’

Cell_line_RMA_proc_ 100%[===================>] 136.15M  18.3MB/s    in 8.3s    

2025-04

# MUTATION DATA

In [ ]:
!wget https://cog.sanger.ac.uk/cmp/download/mutations_20191101.zip

--2025-04-19 00:43:07--  https://cog.sanger.ac.uk/cmp/download/mutations_20191101.zip
Resolving cog.sanger.ac.uk (cog.sanger.ac.uk)... 193.62.203.61, 193.62.203.62, 193.62.203.63
Connecting to cog.sanger.ac.uk (cog.sanger.ac.uk)|193.62.203.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20236900 (19M) [binary/octet-stream]
Saving to: ‘mutations_20191101.zip’

mutations_20191101. 100%[===================>]  19.30M  11.0MB/s    in 1.8s    

2025-04-19 00:43:10 (11.0 MB/s) - ‘mutations_20191101.zip’ saved [20236900/20236900]



In [ ]:
import zipfile
zippy = zipfile.ZipFile("/content/Cell_line_RMA_proc_basalExp.txt.zip")
zippy.extractall("/content/")
zippy.close()

In [ ]:
import zipfile
zippy = zipfile.ZipFile("/content/mutations_20191101.zip")
zippy.extractall("/content/")
zippy.close()

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
import pandas as pd

cell_details = pd.read_excel('/content/Cell_Lines_Details.xlsx')
cell_details.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sample Name,COSMIC identifier,Whole Exome Sequencing (WES),Copy Number Alterations (CNA),Gene Expression,Methylation,Drug\nResponse,GDSC\nTissue descriptor 1,GDSC\nTissue\ndescriptor 2,Cancer Type\n(matching TCGA label),Microsatellite \ninstability Status (MSI),Screen Medium,Growth Properties
0,A253,906794.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,NaN,MSS/MSI-L,D/F12,Adherent
1,BB30-HNC,753531.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
2,BB49-HNC,753532.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
3,BHY,753535.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent
4,BICR10,1290724.0,Y,Y,Y,Y,Y,aero_dig_tract,head and neck,HNSC,MSS/MSI-L,D/F12,Adherent


In [ ]:
luad_cells = cell_details[cell_details['GDSC\nTissue\ndescriptor 2'].str.contains('lung', case=False, na=False)]
luad_cell_names = luad_cells['COSMIC identifier'].astype(str).tolist()

In [ ]:
luad_cell_names = [luad_cell_name.split(".")[0] for luad_cell_name in luad_cell_names]

In [ ]:
drug_response = pd.read_excel('drug_response_data.xlsx')
drug_response

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC1,342,15580432,684057,ES5,SIDM00263,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.966813,0.985678,0.026081,1.299144
1,GDSC1,342,15580806,684059,ES7,SIDM00269,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.692090,0.972690,0.110059,0.156076
2,GDSC1,342,15581198,684062,EW-11,SIDM00203,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.477990,0.944459,0.087019,-0.035912
3,GDSC1,342,15581542,684072,SK-ES-1,SIDM01111,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.033564,0.950758,0.016290,-0.434437
4,GDSC1,342,15581930,687448,COLO-829,SIDM00909,SKCM,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.966007,0.954778,0.180255,0.401702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333156,GDSC1,342,15911377,1659823,SNU-1040,SIDM00217,COREAD,1531,I-CBP112,"EP300, CBP",Chromatin histone acetylation,1005,Y,0.039063,10.0,5.085294,0.972251,0.040661,0.860626
333157,GDSC1,342,15912122,1660035,SNU-61,SIDM00194,COREAD,1531,I-CBP112,"EP300, CBP",Chromatin histone acetylation,1005,Y,0.039063,10.0,5.725399,0.976109,0.045453,1.785602
333158,GDSC1,342,15912431,1660036,SNU-81,SIDM00193,COREAD,1531,I-CBP112,"EP300, CBP",Chromatin histone acetylation,1005,Y,0.039063,10.0,4.930753,0.970851,0.038612,0.637308
333159,GDSC1,342,15912739,1674021,SNU-C5,SIDM00498,COREAD,1531,I-CBP112,"EP300, CBP",Chromatin histone acetylation,1005,Y,0.039063,10.0,4.551784,0.972330,0.042649,0.089683


In [ ]:
erlotinib_data = drug_response[drug_response['DRUG_NAME'] == 'Erlotinib']
erlotinib_luad = erlotinib_data[erlotinib_data['COSMIC_ID'].astype(str).isin(luad_cell_names)]
erlotinib_luad

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
8,GDSC1,342,15587948,687600,NCI-H720,SIDM01120,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.694213,0.986085,0.026592,1.054699
9,GDSC1,342,15590086,687799,NCI-H1648,SIDM00746,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,-3.130315,0.349972,0.044001,-5.064987
10,GDSC1,342,15590484,687800,NCI-H1650,SIDM00745,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.661843,0.983298,0.033401,1.025671
11,GDSC1,342,15591111,687804,NCI-H1770,SIDM00737,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.259282,0.988218,0.025678,0.664688
12,GDSC1,342,15591503,687807,NCI-H1838,SIDM00769,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.322838,0.931209,0.057085,-0.175040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,GDSC1,342,15844397,1240193,NCI-H847,SIDM01149,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.309382,0.987331,0.033730,-0.187106
347,GDSC1,342,15863009,1297439,COR-L95,SIDM00521,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.670089,0.986221,0.037374,1.033066
351,GDSC1,342,15872785,1298352,NCI-H2135,SIDM00700,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.347323,0.988785,0.021415,0.743636
355,GDSC1,342,15883878,1303901,HCC-33,SIDM01071,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.162918,0.987454,0.027080,0.578276


In [ ]:
threshold = erlotinib_luad['LN_IC50'].median()
erlotinib_luad['IC50_class'] = (erlotinib_luad['LN_IC50'] < threshold).astype(int)

<ipython-input-11-84233e8597c6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erlotinib_luad['IC50_class'] = (erlotinib_luad['LN_IC50'] < threshold).astype(int)


In [ ]:
erlotinib_luad

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,IC50_class
8,GDSC1,342,15587948,687600,NCI-H720,SIDM01120,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.694213,0.986085,0.026592,1.054699,0
9,GDSC1,342,15590086,687799,NCI-H1648,SIDM00746,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,-3.130315,0.349972,0.044001,-5.064987,1
10,GDSC1,342,15590484,687800,NCI-H1650,SIDM00745,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.661843,0.983298,0.033401,1.025671,0
11,GDSC1,342,15591111,687804,NCI-H1770,SIDM00737,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.259282,0.988218,0.025678,0.664688,0
12,GDSC1,342,15591503,687807,NCI-H1838,SIDM00769,LUAD,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.322838,0.931209,0.057085,-0.175040,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,GDSC1,342,15844397,1240193,NCI-H847,SIDM01149,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.309382,0.987331,0.033730,-0.187106,1
347,GDSC1,342,15863009,1297439,COR-L95,SIDM00521,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.670089,0.986221,0.037374,1.033066,0
351,GDSC1,342,15872785,1298352,NCI-H2135,SIDM00700,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.347323,0.988785,0.021415,0.743636,0
355,GDSC1,342,15883878,1303901,HCC-33,SIDM01071,SCLC,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.162918,0.987454,0.027080,0.578276,0


In [ ]:
erlotinib_luad['IC50_class'].value_counts()

,count
IC50_class,
0,32
1,31


In [ ]:
expr = pd.read_csv('/content/Cell_line_RMA_proc_basalExp.txt', sep='\t', index_col="GENE_SYMBOLS", header=0)

In [ ]:
expr

,GENE_title,DATA.906826,DATA.687983,DATA.910927,DATA.1240138,DATA.1240139,DATA.906792,DATA.910688,DATA.1240135,DATA.1290812,...,DATA.753584,DATA.907044,DATA.998184,DATA.908145,DATA.1659787,DATA.1298157,DATA.1480372,DATA.1298533,DATA.930299,DATA.905954.1
GENE_SYMBOLS,,,,,,,,,,,,,,,,,,,,,
TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:11858],7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
TNMD,tenomodulin [Source:HGNC Symbol;Acc:17757],2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
DPM1,dolichyl-phosphate mannosyltransferase polypep...,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
SCYL3,SCY1-like 3 (S. cerevisiae) [Source:HGNC Symbo...,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
C1orf112,chromosome 1 open reading frame 112 [Source:HG...,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,hsa-mir-5195 [Source:miRBase;Acc:MI0018174],2.852537,2.776771,2.685307,3.436412,2.951270,3.233383,3.810246,2.792116,2.641117,...,2.699663,5.190438,3.253381,3.000088,2.846830,2.959009,2.974475,2.903894,2.857956,3.033662
POLRMTP1,polymerase (RNA) mitochondrial (DNA directed) ...,3.130696,3.260982,3.176239,3.074432,3.213545,3.382112,3.200106,2.829053,3.158745,...,2.773728,2.988250,3.514337,3.254306,3.139208,3.007502,3.088841,2.847505,2.832840,2.817057
UBL5P2,ubiquitin-like 5 pseudogene 2 [Source:HGNC Sym...,9.986616,9.002814,9.113243,9.958284,9.938978,8.714820,9.396484,9.779745,9.477582,...,9.593772,9.506062,9.945730,9.890244,10.018968,9.332193,10.742651,8.544696,9.900550,9.071943


In [ ]:
expr.columns

Index(['GENE_title', 'DATA.906826', 'DATA.687983', 'DATA.910927',
       'DATA.1240138', 'DATA.1240139', 'DATA.906792', 'DATA.910688',
       'DATA.1240135', 'DATA.1290812',
       ...
       'DATA.753584', 'DATA.907044', 'DATA.998184', 'DATA.908145',
       'DATA.1659787', 'DATA.1298157', 'DATA.1480372', 'DATA.1298533',
       'DATA.930299', 'DATA.905954.1'],
      dtype='object', length=1019)

In [ ]:
expr.drop(["GENE_title"], axis = 1, inplace = True)

In [ ]:
expr

,DATA.906826,DATA.687983,DATA.910927,DATA.1240138,DATA.1240139,DATA.906792,DATA.910688,DATA.1240135,DATA.1290812,DATA.907045,...,DATA.753584,DATA.907044,DATA.998184,DATA.908145,DATA.1659787,DATA.1298157,DATA.1480372,DATA.1298533,DATA.930299,DATA.905954.1
GENE_SYMBOLS,,,,,,,,,,,,,,,,,,,,,
TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,3.455951,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,3.290184,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,11.570155,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,5.560883,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,3.571228,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,2.852537,2.776771,2.685307,3.436412,2.951270,3.233383,3.810246,2.792116,2.641117,3.124607,...,2.699663,5.190438,3.253381,3.000088,2.846830,2.959009,2.974475,2.903894,2.857956,3.033662
POLRMTP1,3.130696,3.260982,3.176239,3.074432,3.213545,3.382112,3.200106,2.829053,3.158745,3.173933,...,2.773728,2.988250,3.514337,3.254306,3.139208,3.007502,3.088841,2.847505,2.832840,2.817057
UBL5P2,9.986616,9.002814,9.113243,9.958284,9.938978,8.714820,9.396484,9.779745,9.477582,9.518999,...,9.593772,9.506062,9.945730,9.890244,10.018968,9.332193,10.742651,8.544696,9.900550,9.071943


In [ ]:
print(expr.columns)

Index(['DATA.906826', 'DATA.687983', 'DATA.910927', 'DATA.1240138',
       'DATA.1240139', 'DATA.906792', 'DATA.910688', 'DATA.1240135',
       'DATA.1290812', 'DATA.907045',
       ...
       'DATA.753584', 'DATA.907044', 'DATA.998184', 'DATA.908145',
       'DATA.1659787', 'DATA.1298157', 'DATA.1480372', 'DATA.1298533',
       'DATA.930299', 'DATA.905954.1'],
      dtype='object', length=1018)


In [ ]:
modified_cols = [c.split(".")[1] for c in expr.columns]
modified_cols

['906826',
 '687983',
 '910927',
 '1240138',
 '1240139',
 '906792',
 '910688',
 '1240135',
 '1290812',
 '907045',
 '906861',
 '906830',
 '909750',
 '1240137',
 '753552',
 '907065',
 '925338',
 '1290809',
 '949158',
 '924110',
 '1240134',
 '907172',
 '687452',
 '905947',
 '1298134',
 '907058',
 '1290814',
 '905975',
 '905939',
 '910704',
 '908460',
 '909745',
 '907173',
 '905936',
 '1303896',
 '907320',
 '688058',
 '905974',
 '907060',
 '908470',
 '908465',
 '906746',
 '909703',
 '724855',
 '724831',
 '906813',
 '946359',
 '909707',
 '910399',
 '687816',
 '907794',
 '909698',
 '910903',
 '910952',
 '909784',
 '724873',
 '684055',
 '906815',
 '906834',
 '906798',
 '910079',
 '908462',
 '1290798',
 '905976',
 '906844',
 '1240212',
 '1298151',
 '906817',
 '1503367',
 '753541',
 '1290730',
 '905977',
 '909255',
 '1240131',
 '1240141',
 '1290810',
 '753618',
 '687829',
 '949155',
 '1290813',
 '909747',
 '1290808',
 '906797',
 '910695',
 '1240132',
 '908150',
 '907169',
 '722045',
 '909723',


In [ ]:
expr.columns = modified_cols

In [ ]:
expr = expr.reset_index().rename(columns={"index":"GENE_SYMBOLS"})

In [ ]:
expr

,GENE_SYMBOLS,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954
0,TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
1,TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
2,DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
3,SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
4,C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17732,NaN,2.852537,2.776771,2.685307,3.436412,2.951270,3.233383,3.810246,2.792116,2.641117,...,2.699663,5.190438,3.253381,3.000088,2.846830,2.959009,2.974475,2.903894,2.857956,3.033662
17733,POLRMTP1,3.130696,3.260982,3.176239,3.074432,3.213545,3.382112,3.200106,2.829053,3.158745,...,2.773728,2.988250,3.514337,3.254306,3.139208,3.007502,3.088841,2.847505,2.832840,2.817057
17734,UBL5P2,9.986616,9.002814,9.113243,9.958284,9.938978,8.714820,9.396484,9.779745,9.477582,...,9.593772,9.506062,9.945730,9.890244,10.018968,9.332193,10.742651,8.544696,9.900550,9.071943
17735,TBC1D3P5,3.073724,3.000182,2.916274,3.256500,3.396126,3.497439,3.193505,3.254539,3.143067,...,3.407260,3.256900,3.189972,3.155584,3.357660,3.435411,3.317945,3.174515,3.243563,3.324517


In [ ]:
expr.fillna("REMOVE", inplace=True)

In [ ]:
expr = expr.drop(expr[expr['GENE_SYMBOLS'] == 'REMOVE'].index)

In [ ]:
expr

,GENE_SYMBOLS,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954
0,TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
1,TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
2,DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
3,SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
4,C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17728,PPY2,2.997054,3.263745,3.112145,3.151576,2.918475,3.141893,3.202237,2.995692,3.038972,...,2.989085,3.113214,2.861577,3.026357,3.217101,3.102979,3.034131,2.981869,2.992148,2.964605
17731,KRT18P55,3.331134,2.992611,2.886574,3.812119,3.412586,2.999640,2.891768,3.457153,2.965871,...,3.257423,3.323432,3.251622,3.041222,3.174122,3.343723,3.412558,3.640390,3.142641,2.927523
17733,POLRMTP1,3.130696,3.260982,3.176239,3.074432,3.213545,3.382112,3.200106,2.829053,3.158745,...,2.773728,2.988250,3.514337,3.254306,3.139208,3.007502,3.088841,2.847505,2.832840,2.817057
17734,UBL5P2,9.986616,9.002814,9.113243,9.958284,9.938978,8.714820,9.396484,9.779745,9.477582,...,9.593772,9.506062,9.945730,9.890244,10.018968,9.332193,10.742651,8.544696,9.900550,9.071943


In [ ]:
expr.columns = expr.columns.str.replace(r'\s*\(.*?\)', '', regex=True)
expr = expr.transpose()

In [ ]:
expr.head()

,0,1,2,3,4,5,6,7,8,9,...,17718,17720,17721,17726,17727,17728,17731,17733,17734,17735
GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
906826,7.632023,2.964585,10.379553,3.614794,3.380681,3.324692,3.56635,8.20453,5.235118,5.369039,...,3.665788,3.134197,4.841169,2.628932,6.786925,2.997054,3.331134,3.130696,9.986616,3.073724
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,3.053174,3.327528,4.570476,2.783441,5.317911,3.263745,2.992611,3.260982,9.002814,3.000182
910927,8.712338,2.643508,9.880733,3.95623,3.23662,3.241246,2.931034,8.191246,5.426841,5.120747,...,3.226808,3.326309,4.214729,2.603604,3.143006,3.112145,2.886574,3.176239,9.113243,2.916274
1240138,7.797142,2.817923,9.883471,4.063701,3.558414,3.101247,7.211707,8.630643,5.617714,4.996434,...,3.110801,2.921903,4.060761,2.61954,3.153896,3.151576,3.812119,3.074432,9.958284,3.2565


In [ ]:
expr.columns = expr.iloc[0]
expr = expr[1:]

In [ ]:
expr

GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
906826,7.632023,2.964585,10.379553,3.614794,3.380681,3.324692,3.56635,8.20453,5.235118,5.369039,...,3.665788,3.134197,4.841169,2.628932,6.786925,2.997054,3.331134,3.130696,9.986616,3.073724
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,3.053174,3.327528,4.570476,2.783441,5.317911,3.263745,2.992611,3.260982,9.002814,3.000182
910927,8.712338,2.643508,9.880733,3.95623,3.23662,3.241246,2.931034,8.191246,5.426841,5.120747,...,3.226808,3.326309,4.214729,2.603604,3.143006,3.112145,2.886574,3.176239,9.113243,2.916274
1240138,7.797142,2.817923,9.883471,4.063701,3.558414,3.101247,7.211707,8.630643,5.617714,4.996434,...,3.110801,2.921903,4.060761,2.61954,3.153896,3.151576,3.812119,3.074432,9.958284,3.2565
1240139,7.729268,2.957739,10.41884,4.3415,3.840373,3.001802,3.375422,8.29695,5.669418,4.180205,...,3.285372,3.474086,4.869199,2.450375,3.65266,2.918475,3.412586,3.213545,9.938978,3.396126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298157,8.441628,2.639276,11.463742,4.425849,4.384732,3.229511,3.571204,8.193,5.6716,4.943996,...,3.390231,3.402212,4.540545,2.595066,5.097882,3.102979,3.343723,3.007502,9.332193,3.435411
1480372,8.422922,2.87989,10.557777,3.55039,4.247189,3.176336,3.321811,8.901706,4.684851,4.215908,...,3.016188,3.841095,4.062441,2.443743,4.243448,3.034131,3.412558,3.088841,10.742651,3.317945
1298533,8.089255,2.521169,10.79275,4.443337,3.071359,3.238305,5.209472,8.073389,5.643811,5.040952,...,4.133042,3.221974,4.68637,2.603842,5.084844,2.981869,3.64039,2.847505,8.544696,3.174515
930299,3.112333,2.870468,9.873902,4.266828,3.230197,3.027742,3.407148,5.76061,5.834256,5.550722,...,2.910977,3.116006,4.099547,2.53128,4.986124,2.992148,3.142641,2.83284,9.90055,3.243563


In [ ]:
print(expr.columns)

Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'LINC00514', 'OR1D5', 'ZNF234', 'MYH4', 'LINC00526', 'PPY2', 'KRT18P55',
       'POLRMTP1', 'UBL5P2', 'TBC1D3P5'],
      dtype='object', name='GENE_SYMBOLS', length=17419)


In [ ]:
expr_luad = expr[expr.index.astype(str).isin(luad_cell_names)]

In [ ]:
expr_luad

GENE_SYMBOLS,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
687983,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,3.053174,3.327528,4.570476,2.783441,5.317911,3.263745,2.992611,3.260982,9.002814,3.000182
908460,8.663931,3.074756,10.163427,4.965831,3.655091,2.99151,7.446631,7.448781,6.541731,5.736827,...,3.019721,3.564249,4.231065,2.738856,5.444887,3.032005,3.377149,3.395202,8.65026,3.310213
908465,8.578107,2.862169,10.150886,5.208023,3.374075,2.984719,7.227592,7.428193,6.803528,5.744833,...,3.310124,3.057923,4.587998,3.052594,6.102892,3.055974,3.049318,3.06334,8.82582,3.37236
724855,6.607496,2.790887,9.572146,4.286477,3.295397,3.114109,3.391469,8.21029,5.650615,4.536686,...,3.42282,3.71579,4.093054,2.69942,5.982371,2.856974,3.270939,3.406193,9.279084,3.184499
724831,6.893817,2.741474,9.878794,3.458342,3.818952,3.081881,3.154842,9.044845,5.647788,5.346201,...,3.313406,3.338737,5.42985,2.708236,5.361093,3.028935,3.132972,3.721237,9.742657,3.523775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330972,8.885371,2.914594,10.355653,5.51554,4.792642,2.993887,3.403733,6.470176,4.741274,5.234,...,3.217882,3.999652,5.319663,2.584468,7.459016,2.933045,3.196616,2.843566,9.297009,3.125922
722046,7.730895,2.691445,10.42001,3.648124,4.008678,3.26019,5.190267,8.477735,7.955872,4.820392,...,3.66976,3.237479,4.302139,2.70568,3.720203,3.063623,3.304472,2.906292,9.825538,3.363418
1298223,7.987541,2.68205,10.499033,4.138102,3.878955,3.262447,3.54029,3.610936,7.345121,5.195728,...,3.334908,3.822109,4.070837,2.628731,4.703836,2.951339,2.806259,3.201912,9.681847,3.356531
905941,5.807695,2.782251,10.283935,3.818307,3.766265,3.153956,3.50484,8.983452,5.900242,4.475349,...,3.258517,3.054645,4.258355,2.563252,3.471069,3.144491,3.823281,2.844305,9.968961,3.182101


In [ ]:
erlotinib_luad['COSMIC_ID'] = erlotinib_luad['COSMIC_ID'].astype(str)
expr_luad.index = expr_luad.index.astype(str)

<ipython-input-34-f5bf2712c0ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erlotinib_luad['COSMIC_ID'] = erlotinib_luad['COSMIC_ID'].astype(str)


In [ ]:
df = erlotinib_luad[['COSMIC_ID', 'IC50_class']].merge(expr_luad, left_on='COSMIC_ID', right_index=True)

In [ ]:
df.head()

,COSMIC_ID,IC50_class,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
8,687600,0,3.30535,2.779445,9.365498,3.748474,3.875906,3.270216,3.139141,5.760702,...,3.304395,3.516952,4.30278,2.580896,6.545751,3.033902,3.081047,3.060162,9.086972,3.474907
9,687799,1,9.115782,2.785111,10.084699,4.236953,3.363104,3.178032,5.936266,8.184636,...,3.223719,3.470181,4.263626,2.42699,5.525464,3.052457,3.691713,3.350326,9.140281,3.102635
10,687800,0,8.19239,2.794146,10.558548,4.479775,3.604665,3.1057,3.911019,7.824828,...,4.197274,3.462764,4.518031,2.598925,5.223581,2.967359,3.205455,2.957154,9.47968,3.074751
11,687804,0,3.435083,2.890112,10.031164,4.267509,3.938229,3.278518,2.93552,5.607512,...,3.739954,3.477276,4.270975,2.698494,6.901225,2.997993,3.088138,3.370998,9.483714,3.379179
12,687807,1,8.040435,2.845976,9.042845,3.956179,3.952157,3.500092,3.952184,7.869846,...,3.300296,3.296698,4.184621,2.689285,5.978527,3.076564,3.536727,3.529058,8.523184,3.23354


In [ ]:
df.to_csv("erlotinib_ic50.csv")

# XGBOOST

In [ ]:
df = pd.read_csv("/content/erlotinib_ic50.csv", index_col=0)

In [ ]:
df.head()

,COSMIC_ID,IC50_class,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
8,687600,0,3.305350,2.779445,9.365498,3.748474,3.875906,3.270216,3.139141,5.760702,...,3.304395,3.516952,4.302780,2.580896,6.545751,3.033902,3.081047,3.060162,9.086972,3.474907
9,687799,1,9.115782,2.785111,10.084699,4.236953,3.363104,3.178032,5.936266,8.184636,...,3.223719,3.470181,4.263626,2.426990,5.525464,3.052457,3.691713,3.350326,9.140281,3.102635
10,687800,0,8.192390,2.794146,10.558548,4.479775,3.604665,3.105700,3.911019,7.824828,...,4.197274,3.462764,4.518031,2.598925,5.223581,2.967359,3.205455,2.957154,9.479680,3.074751
11,687804,0,3.435083,2.890112,10.031164,4.267509,3.938229,3.278518,2.935520,5.607512,...,3.739954,3.477276,4.270975,2.698494,6.901225,2.997993,3.088138,3.370998,9.483714,3.379179
12,687807,1,8.040435,2.845976,9.042845,3.956179,3.952157,3.500092,3.952184,7.869846,...,3.300296,3.296698,4.184621,2.689285,5.978527,3.076564,3.536727,3.529058,8.523184,3.233540


In [ ]:
df['IC50_class'].value_counts()

,count
IC50_class,
1,30
0,29


In [ ]:
df.pop("COSMIC_ID")

,COSMIC_ID
8,687600
9,687799
10,687800
11,687804
12,687807
13,687983
14,687997
15,688006
16,688007
17,688010


In [ ]:
X = df.drop(columns=['IC50_class'])
y = df['IC50_class']

In [ ]:
X

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
8,3.305350,2.779445,9.365498,3.748474,3.875906,3.270216,3.139141,5.760702,5.330641,6.320830,...,3.304395,3.516952,4.302780,2.580896,6.545751,3.033902,3.081047,3.060162,9.086972,3.474907
9,9.115782,2.785111,10.084699,4.236953,3.363104,3.178032,5.936266,8.184636,7.215587,4.828804,...,3.223719,3.470181,4.263626,2.426990,5.525464,3.052457,3.691713,3.350326,9.140281,3.102635
10,8.192390,2.794146,10.558548,4.479775,3.604665,3.105700,3.911019,7.824828,5.284365,5.093818,...,4.197274,3.462764,4.518031,2.598925,5.223581,2.967359,3.205455,2.957154,9.479680,3.074751
11,3.435083,2.890112,10.031164,4.267509,3.938229,3.278518,2.935520,5.607512,5.469530,5.078672,...,3.739954,3.477276,4.270975,2.698494,6.901225,2.997993,3.088138,3.370998,9.483714,3.379179
12,8.040435,2.845976,9.042845,3.956179,3.952157,3.500092,3.952184,7.869846,6.653217,5.716245,...,3.300296,3.296698,4.184621,2.689285,5.978527,3.076564,3.536727,3.529058,8.523184,3.233540
13,7.548671,2.777716,11.807341,4.066887,3.732485,3.152404,7.827172,6.616972,5.809264,7.209653,...,3.053174,3.327528,4.570476,2.783441,5.317911,3.263745,2.992611,3.260982,9.002814,3.000182
14,7.186424,2.786265,9.465668,4.145890,4.024233,3.164200,3.401219,5.780208,5.694426,5.026821,...,3.157382,3.185220,4.109130,2.589561,6.611012,3.038638,3.238802,3.050167,9.504298,3.423033
15,7.833184,2.993816,9.324020,4.095262,3.714190,3.183966,3.422872,3.401835,4.746126,5.973878,...,3.326293,3.931308,4.487871,2.681646,6.363712,3.198058,3.071625,3.158235,9.005766,3.109883
16,4.090947,2.698868,9.772503,4.616728,4.199886,3.379059,3.677094,3.650918,6.132543,5.759918,...,3.217990,4.195953,4.876050,2.484662,5.101850,3.038360,3.277834,3.117028,10.215894,3.201751
17,8.257638,2.853069,9.864538,4.338158,4.157277,3.101810,3.270314,4.402626,4.588982,5.247115,...,3.217158,3.643414,4.869447,2.413709,6.151454,3.043090,2.989509,3.264839,9.707423,3.597898


In [ ]:
import xgboost
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=25
)

In [ ]:
len(X_test)

12

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1]
}

In [ ]:
model_clf = xgboost.XGBClassifier(
    tree_method="hist",
    device = "cuda",
    eval_metric="logloss",
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

In [ ]:
grid_search = GridSearchCV(
    estimator=model_clf,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=2
)

In [ ]:
grid_search.fit(X_train, y_train)
print(f"FITTING COMPLETED!!!")

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   9.8s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=  10.0s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=100; total time=   9.8s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=200; total time=  12.5s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=200; total time=  12.9s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=200; total time=  12.8s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=  20.8s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=  21.1s
[CV] END ..learning_rate=0.01, max_depth=6, n_estimators=500; total time=  20.9s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  10.1s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=   9.7s
[CV] END ..learning_rate=0.01, max_depth=8, n_es

In [ ]:
# Retrieve the best estimator
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Evaluate performance
print("Best Parameters:", grid_search.best_params_)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_proba))

Best Parameters: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100}
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.50      0.55         6
           1       0.57      0.67      0.62         6

    accuracy                           0.58        12
   macro avg       0.59      0.58      0.58        12
weighted avg       0.59      0.58      0.58        12

ROC-AUC Score: 0.6111111111111112


In [ ]:
import joblib
joblib.dump(best_model, "xgboost_gene_expression.joblib")
print(f"MODEL XGBOOST SAVED!!!")

MODEL XGBOOST SAVED!!!


In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df = pd.read_csv("/content/erlotinib_ic50.csv", index_col=0)

In [ ]:
df.pop("COSMIC_ID")

,COSMIC_ID
8,687600
9,687799
10,687800
11,687804
12,687807
13,687983
14,687997
15,688006
16,688007
17,688010


In [ ]:
X = df.drop(columns=['IC50_class'])
y = df['IC50_class']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import xgboost
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=25
)

# Hybrid Classical Quantum Model

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

X_test

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,LINC00514,OR1D5,ZNF234,MYH4,LINC00526,PPY2,KRT18P55,POLRMTP1,UBL5P2,TBC1D3P5
0,6.314431,2.708140,9.648979,4.359162,3.857589,3.165759,3.264701,5.327633,5.154932,4.359228,...,3.392625,3.767117,3.899652,2.711286,6.331357,3.040393,3.641071,3.210252,9.067995,3.236464
1,6.746158,2.760787,10.017339,3.610113,3.495124,3.171601,3.274324,3.482156,4.560099,5.750129,...,3.127766,3.622300,4.274684,2.544405,5.383703,2.915591,3.035224,2.668068,10.005217,3.322755
2,7.186424,2.786265,9.465668,4.145890,4.024233,3.164200,3.401219,5.780208,5.694426,5.026821,...,3.157382,3.185220,4.109130,2.589561,6.611012,3.038638,3.238802,3.050167,9.504298,3.423033
3,8.539394,2.960713,10.378896,4.386587,3.720604,4.054682,5.458658,8.151957,5.881994,4.945048,...,4.902739,3.665562,4.313437,2.617708,4.009441,3.138979,3.122011,2.999795,9.670125,3.506689
4,8.409077,2.804913,9.608832,3.354127,3.387227,3.327735,3.250859,8.010320,8.367287,6.317642,...,3.336778,3.519363,5.422676,2.766779,4.281184,3.264799,3.324969,3.326698,9.337732,3.381863
5,7.510331,2.683506,10.079245,3.858747,3.266789,2.937623,3.133821,5.762559,6.018058,4.563701,...,3.170495,3.858186,4.230431,2.576953,6.510946,2.838603,3.325907,3.008386,9.898382,3.476219
6,7.321865,2.720055,9.025393,3.948645,3.058993,3.263744,9.759216,9.290492,7.157417,4.518753,...,3.312620,4.174488,4.838298,2.606787,5.513134,3.251039,3.445876,3.116187,9.697924,3.157270
7,8.166983,2.889428,9.447908,4.864755,4.819538,3.366459,3.393399,8.109443,5.601087,6.870452,...,3.522697,3.714466,4.004013,2.650775,6.119656,3.228182,2.983671,3.357738,10.094148,3.161363
8,6.194853,2.903636,9.682478,4.325460,4.329243,3.196507,3.406238,6.219118,7.238378,5.100578,...,3.284079,3.353470,4.820576,2.658918,5.957700,3.329993,3.000707,3.230660,8.996333,3.089727
9,3.260012,2.891350,9.589737,3.982191,3.591437,3.149769,3.021613,3.396313,5.497538,5.643947,...,3.643543,3.381757,4.722975,2.694025,5.956700,3.325220,3.297980,3.206651,9.454445,3.301037


In [ ]:
# !pip install pennylane

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_test

array([[ 0.46372814,  0.23108483,  0.74563605, ...,  0.50672333,
         0.37327434,  0.25545538],
       [ 0.53026152,  0.31101324,  0.78904744, ..., -0.05556968,
         0.82318016,  0.34874884],
       [ 0.59811072,  0.34969402,  0.72403279, ...,  0.34070065,
         0.58271779,  0.4571645 ],
       ...,
       [-0.00698703,  0.50923374,  0.73865426, ...,  0.5029891 ,
         0.55878663,  0.32526854],
       [ 0.37656942,  0.52577627,  0.79950009, ...,  0.50688469,
         0.31107769,  0.41258904],
       [ 0.89544315,  0.34794167,  0.79698584, ...,  0.65199216,
         0.40797455,  0.11076647]])

In [ ]:
print(np.unique(y_train), np.unique(y_test))

[0 1] [0 1]


In [ ]:
# !pip install pennylane-lightning[gpu]

In [ ]:
import pennylane as qml
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
# !pip install cuquantum-cu12

In [ ]:
import pennylane as qml

def create_qnode(n_qubits):
    dev = qml.device("lightning.gpu", wires=n_qubits)

    @qml.qnode(dev, interface="torch", diff_method="adjoint") #backprop
    def circuit(inputs, weights):
        for i in range(n_qubits):
            qml.RY(inputs[i], wires=i)
        qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

    return circuit


In [ ]:
import torch
import torch.nn as nn

class HybridQNN(nn.Module):
    def __init__(self, input_dim, n_qubits=4, n_hidden=128, lr=0.01, epochs=20):
        super().__init__()
        self.compress = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            # nn.Dropout(0.3),
            nn.Linear(128, n_qubits),
            nn.Tanh()
        )

        self.qnode = create_qnode(n_qubits)
        weight_shapes = {"weights": (2, n_qubits)}
        self.q_layer = qml.qnn.TorchLayer(self.qnode, weight_shapes)

        self.fc = nn.Linear(n_qubits, 1)

    def forward(self, x):
        x = self.compress(x)
        q_out = torch.stack([self.q_layer(xi) for xi in x], dim=0)
        return self.fc(q_out)


In [ ]:
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
cv_aucs = []

In [ ]:
from logging import log
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

X_data = X_train.copy()
y_data = y_train.copy()

device = "cuda" if torch.cuda.is_available() else "cpu"

def objective(trial):
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    n_qubits = trial.suggest_categorical("n_qubits", [8])
    n_hidden = trial.suggest_categorical("n_hidden", [64, 128])

    max_epochs = 50
    patience = 3

    aucs = []
    skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

    for train_idx, val_idx in skf.split(X_data, y_data):
        model = HybridQNN(input_dim=X_data.shape[1],
                          n_qubits=n_qubits,
                          n_hidden=n_hidden,
                          lr=lr,
                          epochs=max_epochs).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.BCELoss()

        # X_train = torch.tensor(X_data.iloc[train_idx].values).float().to(device)
        # y_train = torch.tensor(y_data.iloc[train_idx].values).float().to(device)

        X_train = torch.from_numpy(X_data[train_idx]).float().to(device)
        y_train = torch.tensor(y_data.iloc[train_idx].values).float().to(device)

        X_val = torch.from_numpy(X_data[val_idx]).float().to(device)
        y_val = torch.tensor(y_data.iloc[val_idx].values).float().to(device)

        best_auc = 0
        no_improve = 0

        for _ in range(max_epochs):
            model.train()
            optimizer.zero_grad()
            out = model(X_train).squeeze()
            loss = criterion(out, y_train)
            loss.backward()
            optimizer.step()

            with torch.no_grad():
                preds = model(X_val).cpu().numpy().squeeze()
                auc = roc_auc_score(y_val.cpu().numpy(), preds)

            if auc > best_auc:
                best_auc = auc
                no_improve = 0
            else:
                no_improve += 1

            if no_improve >= patience:
                break  # Early stopping

        aucs.append(auc)

    return np.mean(aucs)


In [ ]:
import optuna
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=3))

study.optimize(objective, n_trials=20, timeout=5400, show_progress_bar=False)

[I 2025-04-19 04:55:10,098] A new study created in memory with name: no-name-8822f58a-1e5b-4b49-9921-ca8ba82fd59c
[I 2025-04-19 04:55:39,859] Trial 0 finished with value: 0.33933080808080807 and parameters: {'lr': 0.001120229402079025, 'n_qubits': 8, 'n_hidden': 64}. Best is trial 0 with value: 0.33933080808080807.
[I 2025-04-19 04:56:16,192] Trial 1 finished with value: 0.5441919191919192 and parameters: {'lr': 0.0006695683556489702, 'n_qubits': 8, 'n_hidden': 128}. Best is trial 1 with value: 0.5441919191919192.
[I 2025-04-19 04:56:52,843] Trial 2 finished with value: 0.571969696969697 and parameters: {'lr': 0.004172888661692257, 'n_qubits': 8, 'n_hidden': 64}. Best is trial 2 with value: 0.571969696969697.
[I 2025-04-19 04:57:39,155] Trial 3 finished with value: 0.8134469696969697 and parameters: {'lr': 0.0005893185034796475, 'n_qubits': 8, 'n_hidden': 64}. Best is trial 3 with value: 0.8134469696969697.
[I 2025-04-19 04:58:25,671] Trial 4 finished with value: 0.7851957070707071 and

In [ ]:
print("✅ Best AUC:", study.best_value)
print("📦 Best Params:", study.best_params)

✅ Best AUC: 0.8421717171717171
📦 Best Params: {'lr': 0.0002672928751434136, 'n_qubits': 8, 'n_hidden': 128}


In [ ]:
best_params = {'lr': 0.0002672928751434136, 'n_qubits': 8, 'n_hidden': 128}
max_epochs = 25
device = "cuda"


model = HybridQNN(
        input_dim=X_train.shape[1],
        n_qubits=best_params['n_qubits'],
        n_hidden=best_params['n_hidden'],
        lr=best_params['lr'],
        epochs=max_epochs
    ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'])
criterion = nn.BCEWithLogitsLoss()

X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.tensor(y_train.values).float().to(device)

X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.tensor(y_test.values).float().to(device)

for _ in range(max_epochs):
  model.train()
  optimizer.zero_grad()
  out = model(X_train).squeeze()
  loss = criterion(out, y_train)
  loss.backward()
  optimizer.step()

In [ ]:
torch.save(model.state_dict(), "hybrid_qnn_best_model.pt")

In [ ]:
import torch

# Ensure the model is in evaluation mode
model.eval()

# Generate raw outputs (logits)
with torch.no_grad():
    outputs = model(X_test)

# Apply sigmoid to get probabilities
probs = torch.sigmoid(outputs).cpu().numpy()

# Convert probabilities to binary predictions using 0.5 threshold
y_pred = (probs >= 0.5).astype(int)


In [ ]:
probs

array([[0.50940126],
       [0.47572955],
       [0.47925723],
       [0.51612514],
       [0.49370298],
       [0.47977528],
       [0.5143702 ],
       [0.49588883],
       [0.5012308 ],
       [0.49750897],
       [0.48756757],
       [0.5191096 ]], dtype=float32)

In [ ]:
y_true = y_test.cpu().numpy()

In [ ]:
y_true

array([1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1.], dtype=float32)

In [ ]:
y_pred

array([[1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1]])

In [ ]:
print("Classification Report:\n", classification_report(y_true, y_pred))

Classification Report:
               precision    recall  f1-score   support

         0.0       0.71      0.83      0.77         6
         1.0       0.80      0.67      0.73         6

    accuracy                           0.75        12
   macro avg       0.76      0.75      0.75        12
weighted avg       0.76      0.75      0.75        12



In [ ]:
roc_auc_score(y_true, y_pred)

np.float64(0.75)